# Example - Convert dataset to raster (GeoTiff)

Often, it is desirable to take a variable (band) out of your dataset and export it to a raster.
This is possible with the [rio.to_raster()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.to_raster) method. It does most of the work for you so you don't
have to.

Note: The [rio.to_raster()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.to_raster) method only works on a 2-dimensional or 3-dimensional `xarray.DataArray`

In [1]:
import rioxarray
import rasterio
import xarray
import re

See docs for [rioxarray.open_rasterio](../rioxarray.rst#rioxarray-open-rasterio)

In [2]:
rds = rioxarray.open_rasterio(
    "../../test/test_data/input/PLANET_SCOPE_3D.nc",
    variable=["green"],
)
rds

<xarray.Dataset>
Dimensions:      (time: 2, x: 10, y: 10)
Coordinates:
  * time         (time) object 2016-12-19 10:27:29 2016-12-29 12:52:41.659696
  * y            (y) float64 8.085e+06 8.085e+06 ... 8.085e+06 8.085e+06
  * x            (x) float64 4.663e+05 4.663e+05 ... 4.663e+05 4.663e+05
    spatial_ref  int64 0
Data variables:
    green        (time, y, x) float64 ...
Attributes:
    coordinates:  spatial_ref

## Managing Information Loss with xarray operations

Sometimes, you can lose important information from your dataset when performing operations.
You will likely want to keep track of your `nodata` and your `CRS`.

API Reference:

- [rio.to_raster()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.to_raster)
- [rio.write_crs()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.write_crs)
- [rio.update_attrs()](../rioxarray.rst#rioxarray.rioxarray.RasterArray.update_attrs)
- [rio.crs](../rioxarray.rst#rioxarray.rioxarray.RasterArray.crs)
- [rio.nodata](../rioxarray.rst#rioxarray.rioxarray.RasterArray.nodata)

In [3]:
new_ds = rds.green + rds.green
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

({}, CRS.from_epsg(32722), None)

In [4]:
new_ds.rio.to_raster("combination.tif")

In [5]:
!rio info combination.tif

{"bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": ["green", "green"], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["all_valid"], ["all_valid"]], "nodata": null, "res": [3.0, 3.0], "shape": [10, 10], "tiled": false, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}


In [6]:
new_ds = rds.green + rds.green
new_ds.rio.write_crs(rds.green.rio.crs, inplace=True)
new_ds.rio.update_attrs(rds.green.attrs, inplace=True)
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

({'grid_mapping': 'spatial_ref',
  'nodata': 0,
  'units': ('DN', 'DN'),
  '_FillValue': nan,
  'transform': (3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0),
  'scale_factor': 1.0,
  'add_offset': 0.0},
 CRS.from_epsg(32722),
 nan)

In [7]:
new_ds.rio.to_raster("combination_keep_attrs.tif")

In [8]:
!rio info combination_keep_attrs.tif

{"bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": ["green", "green"], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["nodata"], ["nodata"]], "nodata": NaN, "res": [3.0, 3.0], "shape": [10, 10], "tiled": false, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}
